# 16. Apply Model to Dashcam Footage

Given a folder of dashcam footage that has been "split" into images and "metadata.csv" in the previous notebook, process all the images with a detection model.

Outputs:
* A "detections" subdirectory next to the "split" directory of images, containing:
    * "hits" - a directory of images where a bicycle lane marking was found, with an overlay displaying where in the frame and the confidence level
    * "miss" - a directory of images where a bicycle lane marking was NOT found, with overlays for any other object types that were detected
    * "detection_log.csv" - a CSV with details of the frames where a bicycle lane marking was found, including location information

NOTE: During the experiment, this notebook was used several times:

* The first time, we used the model that had been trained on GSV images as-is, without re-training with any dashcam images

The resulting "hits" images -- including false positives -- were labelled and added to the training dataset, and the model was re-trained with a mix of GSV and dashcam images.

* The second time, we used the model that had been trained on a mix of GSV images and dashcam images

Inspection of the resulting "hits" images showed that we were getting false postives due to things such as white markings on the road to "give way" or indicate a traffic island, or a turning lane.  The images were therefore re-labelled to include additional classes to be ignored.  These deterred the model from e.g. assuming a white turning lane might be most similar to a white bicycle lane marking.

* The third time, we used the model that had been trained on a mix of GSV images and dashcam images with multiple decoy classes, and we applied a mask to ignore parts of the frame other than the left hand side of the road.

Using the dash camera footage, we have potentialy many more images to go on than just Google Street View, and we should usually expect an actual bicycle lane marker to be detected in a sequence of frames that are close to each other in time and space.

Therefore, once we have the usual "detection_log.csv" containing a list of ALL frames where we detected bicycle lane markings, we apply a "filtering" process to filter that list down to ONLY detections where there were a minimum number of "hits" within a distance range.

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Name of the subdirectory containing dashcam footage for an area, split into frame images in a
# "split" subdirectory, with an associated "metadata.csv"
# This subdirectory is assumed to be in the 'data_sources' directory
import_directory = 'dashcam_tour_mount_eliza'
#import_directory = 'dashcam_tour_frankston'

# Confidence threshold to apply with the model.  The model must be at least
# this confident of a detection for it to count
confidence_threshold = 0.60

# Trained detection model name
# You might wish to "freeze" a versioned copy of a model and give it a name
trained_model_name   = 'centernet_hg104_512x512_coco17_tpu-8'
#trained_model_name   = 'centernet_V1' # Trained on GSV only
#trained_model_name   = 'centernet_V2' # Trained on GSV plus some dashcam footage
#trained_model_name   = 'centernet_V3' # Trained on CSV plus dashcam with false positives (2000 steps)
#trained_model_name   = 'centernet_V4' # Add RoadDefect and RoadWriting classes to model (2000 steps)
#trained_model_name   = 'centernet_V5' # V4 but with 30,000 steps

# Prefix that will be included as a suffix in the label map file and tfrecord train and test files
dataset_version = 'V1'
#dataset_version = 'V2'

# Detection Mask.  An optional array of pixel coordinates within the frame to include.
# Output images will show this outline as a blue overlay.
# If this is set to None, there is no mask.
# The example mask below excludes the bonnet of the car (which can sometimes cause false-positives
# due to reflections) and the right-hand side of the road
#mask = None
mask = [[0, 0], [0, 720], [480, 650], [950, 650], [950, 0]]

# When applying a filter to "detection_log.csv" to reduce it down to "detection_log_filtered.csv"
# to eliminate "lonely" outlier detections that are not supported by adjacent detections:
min_hits_in_range    = 2   # There must be a minimum of this many hits within a distance range
min_range_from_hit   = 10  # Other hits must be at least this many metres away to count as support
max_range_from_hit   = 50  # Other hits that are more than this many metres away do not count

# E.g. to include the detection in the filtered list, there must be at least two other hits within
# a range >= 10m and <=50m of the original hit for it to count.
# If the vehicle is stationary, giving way to traffic, then the minimum 10m will ensure that these
# effectively "identical" images do not artifically support each other.

## Code

In [2]:
# General imports
import os
import sys
from pathlib import Path

import numpy as np

from ipyleaflet import Map, GeoJSON

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import tf2_utils.tf2_model_wrapper as tf2_model_wrapper
import osm_gsv_utils.detection_map as detection_map
import osm_gsv_utils.detection_log_filter as detection_log_filter

%matplotlib inline

In [3]:
# Derive paths

download_directory = os.path.join(module_path_root, 'data_sources', import_directory, 'split')
batch_filename     = os.path.join(module_path_root, 'data_sources', import_directory, 'split', 'metadata.csv')
output_directory   = os.path.join(module_path_root, 'data_sources', import_directory, 'detections')

print('Processing batch file: [{0:s}]'.format(batch_filename))

# Change directory to make sure the detection model dependencies are found
os.chdir(Path(module_path_root).parent.absolute())

Processing batch file: [E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\split\metadata.csv]


In [4]:
# If a mask was specified, initliase a numpy version of it

if mask is not None:
    mask_np = np.array(mask)
else:
    mask_np = None

In [5]:
# Initialise model
model_wrapper = tf2_model_wrapper(
    import_directory, 
    0, 
    download_directory, 
    output_directory, 
    trained_model_name,
    version_suffix = dataset_version
)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Output directory for detections: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections
Label Map Path: [TensorFlow\workspace\annotations\label_map_V1.pbtxt]
Latest Checkpoint: ckpt-18


In [6]:
# Run detection for entire batch
detection_log = model_wrapper.process_split_dir(
    batch_filename,
    min_score = confidence_threshold,
    mask      = mask_np,
    progress  = True,
    verbose   = False
)

  0%|          | 0/13545 [00:00<?, ?it/s]

Instructions for updating:
Use `tf.cast` instead.


In [7]:
# Create a filtered version of "detection_log.csv" called "detection_log_filtered.csv"
# according to the configured rules
filter = detection_log_filter(os.path.join(output_directory, 'detection_log.csv'))

filter.apply_filter(
    os.path.join(output_directory, 'detection_log_filtered.csv'),
    min_hits_in_range  = 2,
    min_range_from_hit = 5,
    max_range_from_hit = 200
)

  0%|          | 0/232 [00:00<?, ?it/s]